Испортируем библиотеки

In [21]:
import pandas as pd
import numpy as np
from scipy import stats
from math import sqrt

Загружаем датасет и смотрим на первые строчки в данных

In [22]:
df = pd.read_csv('data/marketing_AB.csv', index_col=0)
df.head()

,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


Смотрим структуру датасета, наличие пропусков

In [23]:
display(df.info())
display(df.describe())

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB


None

,user id,total ads,most ads hour
count,5.881010e+05,588101.000000,588101.000000
mean,1.310692e+06,24.820876,14.469061
std,2.022260e+05,43.715181,4.834634
min,9.000000e+05,1.000000,0.000000
25%,1.143190e+06,4.000000,11.000000
50%,1.313725e+06,13.000000,14.000000
75%,1.484088e+06,27.000000,18.000000
max,1.654483e+06,2065.000000,23.000000


Преобразуем столбцы object в category, для оптимизации работы

In [24]:
df = df.astype({
    'user id': 'int64',
    'test group': 'category',
    'converted': 'bool',
    'total ads': 'int64',
    'most ads day': 'category',
    'most ads hour': 'int64'
})
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   user id        588101 non-null  int64   
 1   test group     588101 non-null  category
 2   converted      588101 non-null  bool    
 3   total ads      588101 non-null  int64   
 4   most ads day   588101 non-null  category
 5   most ads hour  588101 non-null  int64   
dtypes: bool(1), category(2), int64(3)
memory usage: 19.6 MB


Проверяем есть ли пользователи, которые в процессе A/Bтеста попали в обе группы. 

In [25]:
group_counts = df.groupby('user id')['test group'].nunique()
ambiguous_users = group_counts[group_counts > 1].index.tolist()
print("\nПользователи, попавшие в обе группы:", ambiguous_users)



Пользователи, попавшие в обе группы: []


Расчет вспомогательных показателей для контрольной и тестовой групп

In [26]:
grouped_count = df.groupby('test group').agg(
    visits=('user id', 'count'),
    purchases=('converted', 'sum'),
)
grouped_count

/tmp/ipykernel_5663/936946756.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_count = df.groupby('test group').agg(


,visits,purchases
test group,,
ad,564577,14423
psa,23524,420


Количество посещений.
— Группа ad: 564 577 пользователей.
— Группа psa: 23 524 пользователей.
Разница в масштабах выборок очень большая (почти в 24 раза), что говорит о неравномерном распределении трафика между вариантами страницы. Выборки не сбалансированы по численности, и прямые сравнения требуют осторожности или дополнительного стратифицированного анализа.

Суммарное число покупок.
— В «ad» совершено 14 423 покупок, в «psa» — 420. Опять же численность продаж пропорциональна числу визитов.

Ключевые показатели по группам:

In [27]:
grouped_conv = df.groupby('test group').agg(
    avg_total_ads=('total ads', 'mean')
)
grouped_conv['conversion_rate'] = grouped_count['purchases'] / grouped_count['visits']
grouped_conv


/tmp/ipykernel_5663/1885145049.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_conv = df.groupby('test group').agg(


,avg_total_ads,conversion_rate
test group,,
ad,24.823365,0.025547
psa,24.761138,0.017854


Ключевые показатели по группам:

Конверсия (доля совершивших покупку):
— «ad»: 2.56 %
— «psa»: 1.79 %

Среднее число показов рекламы (total ads):
— «ad»: 24.82 показы на пользователя.
— «psa»: 24.76 показы (поскольку пользователи в группе «psa» технически не видели «рекламу», это может быть шум/метрики фоновых объявлений).

Первичные выводы:

Вариант А («ad») демонстрирует более высокую конверсию (2.56 % против 1.79 %). По показателю конверсии он выглядит эффективнее.

Среднее число рекламы практически одинаково в обеих группах (≈24.8), поэтому разница в конверсии не объясняется объёмом увиденных объявлений.

На первый взгляд, вариант A (ad) выгоднее по конверсии. Однако для окончательных выводов нужно учесть несбалансированность выборок, провести статистическую проверку значимости (например, z-тест пропорций) и, возможно, скорректировать анализ с помощью взвешивания или бутстрэппинга.

Формулировка гипотез и выбор тестов

Нулевая гипотеза - между группами нет разницы — они работают одинаково.
Альтернативная гипотеза - между группами есть разница — одна рекламная компания лучше другой.

Z‑тест для долей подходит для проверки разницы в процентах (конверсии) на очень больших выборках.

Welch’s t‑тест выбран для сравнения средних показов рекламы, так как он не требует равенства дисперсий и хорошо работает при большом объёме данных.

In [28]:
ad = df[df['test group'] == 'ad']
psa = df[df['test group'] == 'psa']

n1, x1 = len(ad), ad['converted'].sum()
n2, x2 = len(psa), psa['converted'].sum()
p1, p2 = x1/n1, x2/n2
p_pool = (x1 + x2) / (n1 + n2)
se = sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2))
z, p_val = (p1-p2)/se, 2*(1-stats.norm.cdf(abs((p1-p2)/se)))
print(f"z = {z:.2f}, p-value = {p_val:.3e}")

z = 7.37, p-value = 1.705e-13


p-value очень маленькое (почти 0) → разница в конверсии не случайна. «ad» лучше.

Доверительный интервал для разницы конверсий

In [29]:
# 95% интервал разницы
z_crit = stats.norm.ppf(0.975)
se_diff = sqrt(p1*(1-p1)/n1 + p2*(1-p2)/n2)
ci_low = (p1-p2) - z_crit*se_diff
ci_high = (p1-p2) + z_crit*se_diff
print(f"95% CI разницы: [{ci_low:.3%}, {ci_high:.3%}]")

95% CI разницы: [0.595%, 0.943%]


Разница в конверсии точно лежит между ~0.6 % и ~0.9 % в пользу «ad».

Проверка разницы среднего числа рекламы

In [30]:
# Берём выборки объявлений
ads1, ads2 = ad['total ads'], psa['total ads']
# Проверка формы распределения на небольших подвыборках
sw1 = stats.shapiro(ads1.sample(500))
sw2 = stats.shapiro(ads2.sample(500))
print("Shapiro p-values:", sw1.pvalue, sw2.pvalue)
# Разница средних: Welch’s t-test
t_stat, p_t = stats.ttest_ind(ads1, ads2, equal_var=False)
print(f"t = {t_stat:.2f}, p-value = {p_t:.3e}")

Shapiro p-values: 1.088683823305908e-34 1.1792116930269707e-29
t = 0.22, p-value = 8.274e-01


Распределение не идеально «колокольное», но у нас много данных, поэтому t-тест подойдёт. p-value примерно 0.8 → разницы нет.


Доверительный интервал для разницы средних рекламы.

In [31]:
m1, m2 = ads1.mean(), ads2.mean()
se_diff_ads = sqrt(ads1.std(ddof=1)**2/n1 + ads2.std(ddof=1)**2/n2)
ci_low_ad = (m1-m2) - z_crit*se_diff_ads
ci_high_ad = (m1-m2) + z_crit*se_diff_ads
print(f"95% CI разницы средних: [{ci_low_ad:.3f}, {ci_high_ad:.3f}]")

95% CI разницы средних: [-0.497, 0.622]


Среднее число объявлений почти не отличается. Диапазон разницы включает 0.

Выводы, которые мы можем сделать на основе статистических тестов

Реклама повышает конверсию. Группа «ad» показывает на 0.6–0.94 п.п. более высокую конверсию, и эта разница статистически надёжна.

Эффект не обусловлен количеством показов. Среднее число показов в обеих группах практически одинаково, и разница незначима.

Инвестировать в рекламный вариант («ad»), поскольку он повышает долю покупателей без увеличения числа показов — следовательно, дополнительная вовлечённость и доход от рекламы очевидны.

